# Imports and Data Download :

In [59]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

print(torch.__version__)

mnist_trainset = torchvision.datasets.MNIST(
                              root='./data', 
                              train=True, 
                              download=True, 
                              transform=transforms.Compose([
                                  transforms.ToTensor()
                              ])
                  )

1.9.0+cu111


# Data Preparation :
* This code contains logic to create the 

In [60]:
def create_one_hotencoding_inp(x: int):
  """
  This function allows user to convert the input number (between 0 to 9) 
  in to one hot encoded vector of size 10.
  input : x = number between 0-9
  output : base_arr = one hot encoded vector with 10 list elements
  """
  assert isinstance(x, int) or isinstance(x, np.int64) or isinstance(x, np.int32), "input should be of datatype 'int'"
  assert x>=0 and x<=9, "input should be between 0-9"
  base_arr = np.zeros(10)
  base_arr[x] = 1
  return base_arr

# create_one_hotencoding_inp("2")  # Error checking for string input
# create_one_hotencoding_inp(10) # Error checking for number not in range 0-9

In [61]:
def extract_images_and_labels(num_obs: int, mnist_trainset):
  i=0
  i_max=int(num_obs)
  lst_imgs = []
  lst_labels = []
  for img, label in mnist_trainset:
    lst_imgs.append(img)
    lst_labels.append(label)
    i += 1
    if i==i_max:
      break;

  # Checking class distribution
  df_class_distro = pd.DataFrame(pd.Series(lst_labels).value_counts().sort_index(), columns=['count_labels']).reset_index()
  df_class_distro.rename({'index':'class_label'}, axis=1, inplace=True)
  print(f"Printing class distibution below : \n{df_class_distro}")
  return(lst_imgs, lst_labels)

lst_imgs, lst_labels = extract_images_and_labels(30000, mnist_trainset)
print(len(lst_imgs))
print(len(lst_labels))

Printing class distibution below : 
   class_label  count_labels
0            0          2961
1            1          3423
2            2          2948
3            3          3073
4            4          2926
5            5          2709
6            6          2975
7            7          3107
8            8          2875
9            9          3003
30000
30000


In [62]:
def prepare_my_dataset(lst_imgs: list, lst_labels: list):
  """
  This function will prepare the dataset as per the below requirement :
  Creates the tensor of input images
  Creates the random input numbers same as size of input images & creates the one hot encoded vectors of each obs
  Creates the tensor of output image labels
  Creates the target sum which is sum of input random number and image label
  """
  inp_images = torch.stack(lst_imgs)
  out_img_labels = torch.tensor(lst_labels)

  i = 0
  while i!=10:
    inp_rand_np = np.random.randint(0,10,len(out_img_labels))
    i = pd.Series(inp_rand_np).nunique()
    print(f"Number of unique values in my random set '{i}'")

  inp_rand = torch.tensor(inp_rand_np)
  out_sum = inp_rand + out_img_labels

  inp_rand_ohe = torch.tensor([create_one_hotencoding_inp(x) for x in inp_rand_np])
  return(inp_images, inp_rand_ohe, out_img_labels, out_sum)

inp_images, inp_rand_ohe, out_img_labels, out_sum = prepare_my_dataset(lst_imgs, lst_labels)
print(inp_images.shape)
print(inp_rand_ohe.shape)
print(out_img_labels.shape)
print(out_sum.shape)

Number of unique values in my random set '10'
torch.Size([30000, 1, 28, 28])
torch.Size([30000, 10])
torch.Size([30000])
torch.Size([30000])


In [63]:
from torch.utils.data import Dataset

# Creating the Custom dataset using previously created data so that we can use this dataset object to create batches later
class CustomMnistDataset(Dataset):
  def __init__(self):
    self.data = []
    for i in range(0,len(out_sum)):
      self.data.append([inp_images[i], inp_rand_ohe[i], out_img_labels[i], out_sum[i]])

  def __getitem__(self, index):
    return self.data[index]
    
  def __len__(self):
    return len(self.data)

myDataset = CustomMnistDataset()
len(myDataset)

30000

In [64]:
# creating train_loader so that we can load data in batches while training
train_loader = torch.utils.data.DataLoader(myDataset
    ,batch_size=100
    ,shuffle=True
)

len(train_loader)

300

In [65]:
# Printing the size of first 3 batches
i=0
for tr in train_loader:
  print(tr[0].shape, tr[1].shape, tr[2].shape, tr[3].shape)
  i+=1
  if i==3:
    break;

torch.Size([100, 1, 28, 28]) torch.Size([100, 10]) torch.Size([100]) torch.Size([100])
torch.Size([100, 1, 28, 28]) torch.Size([100, 10]) torch.Size([100]) torch.Size([100])
torch.Size([100, 1, 28, 28]) torch.Size([100, 10]) torch.Size([100]) torch.Size([100])


# Model Building :

In [66]:
import torch.nn.functional as F
import torch.nn as nn

In [67]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    #28 - > 24 - > 12 -> 8 -> 4
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5) 
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
    self.fc1 = nn.Linear(in_features=(12 * 4 * 4)+10, out_features=120)
    self.fc2 = nn.Linear(in_features=120, out_features=60)
    self.out1 = nn.Linear(in_features=60, out_features=10)
    self.out2 = nn.Linear(in_features=60, out_features=19)
  
  def forward(self, t1, t2):
    # input layer
    x, y = t1, t2

    # conv1 layer
    x = self.conv1(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2) # 28 | 24 | 12

    # conv2 layer
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2) # 12 | 8 | 4 >> 12x4x4

    # reshapre
    x = x.reshape(-1, 12 * 4 * 4)
    z = torch.cat((x,y.reshape(-1,10)), dim=1)
    
    # fc1 layer
    z = self.fc1(z)
    z = F.relu(z)

    # fc2 layer
    z = self.fc2(z)
    z = F.relu(z)

    # output layer 1
    o1 = self.out1(z)
    o1 = F.softmax(o1, dim=1)

    # output layer 2
    o2 = self.out2(z)
    o2 = F.softmax(o2, dim=1)
    return o1, o2

In [68]:
network = Network()

print(network)

Network(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=202, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out1): Linear(in_features=60, out_features=10, bias=True)
  (out2): Linear(in_features=60, out_features=19, bias=True)
)


In [69]:
import torch.optim as optim

torch.set_grad_enabled(True)

In [70]:
def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

In [71]:
optimizer = optim.Adam(network.parameters(), lr=0.01)

for epoch in range(10):

    total_loss = 0
    total_correct_labels = 0
    total_correct_sum = 0

    for batch in train_loader: # Get Batch
        i_images, i_rand, o_labels, o_sum = batch 

        preds_labels, preds_sum = network(i_images, i_rand.float()) # Pass Batch
        loss1 = F.cross_entropy(preds_labels, o_labels) # Calculate Loss
        loss2 = F.cross_entropy(preds_sum, o_sum) # Calculate Loss
        loss = loss1 + loss2

        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct_labels += get_num_correct(preds_labels, o_labels)
        total_correct_sum += get_num_correct(preds_sum, o_sum)

    print(
        "epoch", epoch, 
        "total_correct_labels:", total_correct_labels, 
        "total_correct_sum:", total_correct_sum, 
        "loss:", total_loss
    )

epoch 0 total_correct_labels: 24274 total_correct_sum: 2986 loss: 1373.3529806137085
epoch 1 total_correct_labels: 27484 total_correct_sum: 3051 loss: 1342.14302444458
epoch 2 total_correct_labels: 27451 total_correct_sum: 3083 loss: 1342.2798962593079
epoch 3 total_correct_labels: 25971 total_correct_sum: 3041 loss: 1357.4546599388123
epoch 4 total_correct_labels: 23675 total_correct_sum: 3010 loss: 1380.752944469452
epoch 5 total_correct_labels: 26626 total_correct_sum: 3081 loss: 1350.5772757530212
epoch 6 total_correct_labels: 25553 total_correct_sum: 3096 loss: 1361.1561498641968
epoch 7 total_correct_labels: 24234 total_correct_sum: 2999 loss: 1375.3053560256958
epoch 8 total_correct_labels: 26422 total_correct_sum: 3033 loss: 1353.0957169532776
epoch 9 total_correct_labels: 25102 total_correct_sum: 3073 loss: 1365.8992285728455
